In [1]:
import emoji
import pandas as pd
import numpy as np
from keras.layers import *
from keras.models import Sequential
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.
/home/prakhar/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/prakhar/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/prakhar/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/prakhar/.local/lib/python3.6/site-packages/tensorflow/py

In [2]:
emoji.EMOJI_UNICODE

{':1st_place_medal:': '🥇',
 ':2nd_place_medal:': '🥈',
 ':3rd_place_medal:': '🥉',
 ':AB_button_(blood_type):': '🆎',
 ':ATM_sign:': '🏧',
 ':A_button_(blood_type):': '🅰',
 ':Afghanistan:': '🇦🇫',
 ':Albania:': '🇦🇱',
 ':Algeria:': '🇩🇿',
 ':American_Samoa:': '🇦🇸',
 ':Andorra:': '🇦🇩',
 ':Angola:': '🇦🇴',
 ':Anguilla:': '🇦🇮',
 ':Antarctica:': '🇦🇶',
 ':Antigua_&_Barbuda:': '🇦🇬',
 ':Aquarius:': '♒',
 ':Argentina:': '🇦🇷',
 ':Aries:': '♈',
 ':Armenia:': '🇦🇲',
 ':Aruba:': '🇦🇼',
 ':Ascension_Island:': '🇦🇨',
 ':Australia:': '🇦🇺',
 ':Austria:': '🇦🇹',
 ':Azerbaijan:': '🇦🇿',
 ':BACK_arrow:': '🔙',
 ':B_button_(blood_type):': '🅱',
 ':Bahamas:': '🇧🇸',
 ':Bahrain:': '🇧🇭',
 ':Bangladesh:': '🇧🇩',
 ':Barbados:': '🇧🇧',
 ':Belarus:': '🇧🇾',
 ':Belgium:': '🇧🇪',
 ':Belize:': '🇧🇿',
 ':Benin:': '🇧🇯',
 ':Bermuda:': '🇧🇲',
 ':Bhutan:': '🇧🇹',
 ':Bolivia:': '🇧🇴',
 ':Bosnia_&_Herzegovina:': '🇧🇦',
 ':Botswana:': '🇧🇼',
 ':Bouvet_Island:': '🇧🇻',
 ':Brazil:': '🇧🇷',
 ':British_Indian_Ocean_Territory:': '🇮🇴',
 ':British_Virgin_Is

In [3]:
emoji_dict = {"0":"\u2764\uFE0F",
             "1":":baseball:",
             "2":":grinning_face_with_big_eyes:",
             "3":":disappointed_face:",
             "4":":fork_and_knife:"}

In [4]:
for e in emoji_dict.values():
    print(emoji.emojize(e))

❤️
⚾
😃
😞
🍴


## Preprocessing data

In [5]:
train = pd.read_csv("dataset/train_emoji.csv", header=None)
test = pd.read_csv("dataset/test_emoji.csv", header=None)

In [6]:
train.head()

0  1   2     3
0           never talk to me again  3 NaN   NaN
1  I am proud of your achievements  2 NaN   NaN
2   It is the worst day in my life  3 NaN   NaN
3                 Miss you so much  0 NaN   [0]
4                     food is life  4 NaN   NaN

In [7]:
x_train = train[0]
y_train = train[1]

x_test = test[0]
y_test = test[1]

In [8]:
for i in range(5):
    print(x_train[i], emoji.emojize(emoji_dict[str(y_train[i])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


 ## Converting sentences into embeddings

In [9]:
f = open("glove6b50dtxt/glove.6B.50d.txt")

In [10]:
embedding_index = {}

for line in f:

    sent = line.split()
    word = sent[0]
    values = np.asarray(sent[1:], dtype='float')

    embedding_index[word] = values

f.close()

## Converting senteces into vectors

In [11]:
def out(x):
    
    maxlen = 10
    embedding_dim = 50
    embedding_out = np.zeros((x.shape[0], maxlen, embedding_dim))
    
    for ix in range(x.shape[0]):
        x[ix] = x[ix].split()
        
        for ij in range(len(x[ix])):
            try:
                embedding_out[ix][ij] = embedding_index[x[ix][ij].lower()]
            except:
                embedding_out[ix][ij] = np.zeros((50,))
            
    return embedding_out
        
    

In [12]:
train_emb = out(x_train)
test_emb = out(x_test)
print(train_emb.shape, test_emb.shape)

(132, 10, 50) (56, 10, 50)


/home/prakhar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## LSTM architechture

In [13]:
model = Sequential()
model.add(LSTM(64, input_shape=(10, 50)))
model.add(Dropout(.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 29,765
Trainable params: 29,765
Non-trainable params: 0
_________________________________________________________________


In [14]:
y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

In [15]:
checkpoint = ModelCheckpoint("best_model1.h5", monitor="val_loss", verbose=True, save_best_only=True)
earlystop = EarlyStopping(monitor="val_acc", patience=10)

hist = model.fit(train_emb, y_train, epochs=100, batch_size=64, shuffle=True, validation_split=.2)

Instructions for updating:
Use tf.cast instead.
Train on 105 samples, validate on 27 samples
Epoch 1/100
105/105 [==============================] - 2s 19ms/step - loss: 1.6093 - acc: 0.1619 - val_loss: 1.6071 - val_acc: 0.2593
Epoch 2/100
105/105 [==============================] - 0s 425us/step - loss: 1.5956 - acc: 0.2667 - val_loss: 1.6094 - val_acc: 0.1852
Epoch 3/100
105/105 [==============================] - 0s 457us/step - loss: 1.5547 - acc: 0.3333 - val_loss: 1.6131 - val_acc: 0.1852
Epoch 4/100
105/105 [==============================] - 0s 442us/step - loss: 1.5617 - acc: 0.3238 - val_loss: 1.6180 - val_acc: 0.1852
Epoch 5/100
105/105 [==============================] - 0s 437us/step - loss: 1.5249 - acc: 0.3238 - val_loss: 1.6257 - val_acc: 0.1852
Epoch 6/100
105/105 [==============================] - 0s 431us/step - loss: 1.5124 - acc: 0.3333 - val_loss: 1.6357 - val_acc: 0.1852
Epoch 7/100
105/105 [==============================] - 0s 475us/step - loss: 1.5085 - acc: 0.3238 

105/105 [==============================] - 0s 394us/step - loss: 0.1382 - acc: 0.9524 - val_loss: 1.5146 - val_acc: 0.5926
Epoch 60/100
105/105 [==============================] - 0s 385us/step - loss: 0.1062 - acc: 0.9714 - val_loss: 1.6682 - val_acc: 0.5926
Epoch 61/100
105/105 [==============================] - 0s 391us/step - loss: 0.1187 - acc: 0.9714 - val_loss: 1.6663 - val_acc: 0.5926
Epoch 62/100
105/105 [==============================] - 0s 377us/step - loss: 0.0869 - acc: 0.9905 - val_loss: 1.6176 - val_acc: 0.6296
Epoch 63/100
105/105 [==============================] - 0s 375us/step - loss: 0.0974 - acc: 0.9810 - val_loss: 1.5557 - val_acc: 0.6296
Epoch 64/100
105/105 [==============================] - 0s 376us/step - loss: 0.0593 - acc: 0.9905 - val_loss: 1.4795 - val_acc: 0.6296
Epoch 65/100
105/105 [==============================] - 0s 383us/step - loss: 0.0913 - acc: 0.9905 - val_loss: 1.5051 - val_acc: 0.5926
Epoch 66/100
105/105 [==============================] - 0s 37

In [16]:
pred = model.predict_classes(test_emb)

In [17]:
print(pred)

[4 3 3 2 2 2 1 2 4 2 1 2 0 3 1 3 2 2 3 2 0 0 4 2 3 1 2 0 4 2 0 1 3 2 0 2 2
 4 4 2 1 0 0 2 2 2 2 2 3 1 1 0 3 2 2 3]


In [18]:
model.load_weights("best_model1.h5")

OSError: Unable to open file (unable to open file: name = 'best_model1.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
model.evaluate(test_emb, y_test)